## Homework 3: Symbolic Music Generation Using Markov Chains

**Before starting the homework:**

Please run `pip install miditok` to install the [MiDiTok](https://github.com/Natooz/MidiTok) package, which simplifies MIDI file processing by making note and beat extraction more straightforward.

You’re also welcome to experiment with other MIDI processing libraries such as [mido](https://github.com/mido/mido), [pretty_midi](https://github.com/craffel/pretty-midi) and [miditoolkit](https://github.com/YatingMusic/miditoolkit). However, with these libraries, you’ll need to handle MIDI quantization yourself, for example, converting note-on/note-off events into beat positions and durations.

In [8]:
# run this command to install MiDiTok
! pip install miditok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.8 MB/s eta 0:00:00


In [2]:
# comment this out when submitting
!pip install midiutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for midiutil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54569 sha256=a840d036aaa4a5e39e56ca6ed8acac381f004862d6961d2920b8629048252542
  Stored in directory: /root/.cache/pip/wheels/6c/42/75/fce10c67f06fe627fad8acd1fd3a004a24e07b0f077761fbbd
Successfully built midiutil


In [9]:
# import required packages
import random
from glob import glob
from collections import defaultdict

import numpy as np
from numpy.random import choice

from symusic import Score
from miditok import REMI, TokenizerConfig
from midiutil import MIDIFile

In [ ]:
# You can change the random seed but try to keep your results deterministic!
# If I need to make changes to the autograder it'll require rerunning your code,
# so it should ideally generate the same results each time.
random.seed(42)

### Load music dataset
We will use a subset of the [PDMX dataset](https://zenodo.org/records/14984509).

Please find the link in the homework spec.

All pieces are monophonic music (i.e. one melody line) in 4/4 time signature.

In [4]:
# comment this out when submitting
!unzip PDMX_subset.zip

Archive:  PDMX_subset.zip
   creating: PDMX_subset/
  inflating: PDMX_subset/QmWCdYmueNZMNSW4sK3Dd6ogXKjV1RkaUcrmm3nd7iKZAQ.mid  
  inflating: PDMX_subset/QmZZE6Xnfgiw9RgYiCL1h5x3j9jocazqWha5k9LETrGt9B.mid  
  inflating: PDMX_subset/QmYkLzny11AS5tr543YTcrNsLaLaHeQWWTUbHsLpR7nH2g.mid  
  inflating: PDMX_subset/QmWh8zMp4pi6aG4t9i5HgVrzYsJRLdQoZucCkaebfou5qK.mid  
  inflating: PDMX_subset/QmTkcC9pKEUNofoSW4Tuxb7vCeugW7kD3DBUEWkeUoKFPv.mid  
  inflating: PDMX_subset/QmWGxGFTUgaLjkRPVtvNwuELZyRD4TRmmVCfSkrmxiQkNi.mid  
  inflating: PDMX_subset/QmXk4uVrEnJkBZXBr7mQoBeXebmYPtxZGerp5MExpYS3NR.mid  
  inflating: PDMX_subset/QmQ5pMi7ieqoTHghLhjAqSLX5ptY3639ocwGdt7ew9MRhn_0.mid  
  inflating: PDMX_subset/QmNzmh6KezdyMBotBTbHkh1wakhurGb4bQnB7hWhuwRRJ4.mid  
  inflating: PDMX_subset/Qmdatn9HR89XLxeva56MsvMHZqPCZP5vqHiVTJW3i4VZSB_0.mid  
  inflating: PDMX_subset/QmXP8V39GGaXMSM5m19Zx7oEVN9mNpSB6fqVmQt3feetBr.mid  
  inflating: PDMX_subset/QmUsNp1UGDKxb5pHbeK5wxjwV4cakJyciW9nJ4gAoRSSk7.mid  
  inflat

In [5]:
midi_files = glob('PDMX_subset/*.mid')
len(midi_files)

1000

### Train a tokenizer with the REMI method in MidiTok

In [10]:
config = TokenizerConfig(num_velocities=1, use_chords=False, use_programs=False)
tokenizer = REMI(config)
tokenizer.train(vocab_size=1000, files_paths=midi_files)

### Use the trained tokenizer to get tokens for each midi file
In REMI representation, each note will be represented with four tokens: `Position, Pitch, Velocity, Duration`, e.g. `('Position_28', 'Pitch_74', 'Velocity_127', 'Duration_0.4.8')`; a `Bar_None` token indicates the beginning of a new bar.

In [11]:
# e.g.:
midi = Score(midi_files[0])
tokens = tokenizer(midi)[0].tokens
tokens[:10]

['Bar_None',
 'Position_0',
 'Pitch_76',
 'Velocity_127',
 'Duration_0.4.8',
 'Position_4',
 'Pitch_69',
 'Velocity_127',
 'Duration_0.4.8',
 'Position_8']

1. Write a function to extract note pitch events from a midi file; and another extract all note pitch events from the dataset and output a dictionary that maps note pitch events to the number of times they occur in the files. (e.g. {60: 120, 61: 58, …}).

`note_extraction()`
- **Input**: a midi file

- **Output**: a list of note pitch events (e.g. [60, 62, 61, ...])

`note_frequency()`
- **Input**: all midi files `midi_files`

- **Output**: a dictionary that maps note pitch events to the number of times they occur, e.g {60: 120, 61: 58, …}

In [12]:
def note_extraction(midi_file):
    # Q1a: Your code goes here
    midi = Score(midi_file)
    tokens = tokenizer(midi)[0].tokens
    pitch_events = [
        int(token.split('_')[1]) for token in tokens if token.startswith("Pitch_")
    ]
    return pitch_events

In [13]:
def note_frequency(midi_files):
    # Q1b: Your code goes here
    pitch_counts = defaultdict(int)
    for midi_file in midi_files:
        pitch_events = note_extraction(midi_file)
        for pitch_event in pitch_events:
            pitch_counts[pitch_event] += 1
    return pitch_counts

2. Write a function to normalize the above dictionary to produce probability scores (e.g. {60: 0.13, 61: 0.065, …})

`note_unigram_probability()`
- **Input**: all midi files `midi_files`

- **Output**: a dictionary that maps note pitch events to probabilities, e.g. {60: 0.13, 61: 0.06, …}

In [14]:
def note_unigram_probability(midi_files):
    note_counts = note_frequency(midi_files)
    unigramProbabilities = {}

    # Q2: Your code goes here
    total_notes = sum(note_counts.values())
    for note, count in note_counts.items():
        unigramProbabilities[note] = count / total_notes

    return unigramProbabilities

3. Generate a table of pairwise probabilities containing p(next_note | previous_note) values for the dataset; write a function that randomly generates the next note based on the previous note based on this distribution.

`note_bigram_probability()`
- **Input**: all midi files `midi_files`

- **Output**: two dictionaries:

  - `bigramTransitions`: key: previous_note, value: a list of next_note, e.g. {60:[62, 64, ..], 62:[60, 64, ..], ...} (i.e., this is a list of every other note that occured after note 60, every note that occured after note 62, etc.)

  - `bigramTransitionProbabilities`: key:previous_note, value: a list of probabilities for next_note in the same order of `bigramTransitions`, e.g. {60:[0.3, 0.4, ..], 62:[0.2, 0.1, ..], ...} (i.e., you are converting the values above to probabilities)

`sample_next_note()`
- **Input**: a note

- **Output**: next note sampled from pairwise probabilities

In [15]:
from collections import Counter

In [16]:
def note_bigram_probability(midi_files):
    bigramTransitions = defaultdict(list)
    bigramTransitionProbabilities = defaultdict(list)

    # Q3a: Your code goes here
    bigramTransitionsSets = defaultdict(set)
    note_counts = defaultdict(Counter)  # {prev_note: Counter({next_note: count})}
    for midi_file in midi_files:
        pitch_events = note_extraction(midi_file)
        for i in range(len(pitch_events) - 1):
            prev_note = pitch_events[i]
            next_note = pitch_events[i + 1]
            bigramTransitionsSets[prev_note].add(next_note)
            note_counts[prev_note][next_note] += 1
    for (prev_note, next_notes) in bigramTransitionsSets.items():
        bigramTransitions[prev_note] = list(next_notes)

    for prev_note, next_notes in bigramTransitions.items():
        total_notes = sum(note_counts[prev_note].values())
        for next_note in next_notes:
            bigramTransitionProbabilities[prev_note].append(note_counts[prev_note][next_note] / total_notes)

    # old code, this is wrong

    # for midi_file in midi_files:
    #     pitch_events = note_extraction(midi_file)
    #     for i in range(len(pitch_events) - 1):
    #         prev_note = pitch_events[i]
    #         next_note = pitch_events[i + 1]
    #         bigramTransitions[prev_note].append(next_note)

    # for prev_note, next_notes in bigramTransitions.items():
    #     total_notes = len(next_notes)
    #     note_counts = defaultdict(int)
    #     for next_note in next_notes:
    #         note_counts[next_note] += 1
    #     for note in next_notes:
    #         bigramTransitionProbabilities[note] += note_counts[note] / total_notes

    return bigramTransitions, bigramTransitionProbabilities

In [17]:
def sample_next_note(note):
    # Q3b: Your code goes here
    bigramTransitions, bigramTransitionProbabilities = note_bigram_probability(midi_files)
    next_notes = bigramTransitions[note]
    probabilities = bigramTransitionProbabilities[note]
    return random.choices(next_notes, weights=probabilities, k=1)[0]

4. Write a function to calculate the perplexity of your model on a midi file.

    The perplexity of a model is defined as

    $\quad \text{exp}(-\frac{1}{N} \sum_{i=1}^N \text{log}(p(w_i|w_{i-1})))$

    where $p(w_1|w_0) = p(w_1)$, $p(w_i|w_{i-1}) (i>1)$ refers to the pairwise probability p(next_note | previous_note).

`note_bigram_perplexity()`
- **Input**: a midi file

- **Output**: perplexity value

In [18]:
def note_bigram_perplexity(midi_file):
    unigramProbabilities = note_unigram_probability(midi_files)
    bigramTransitions, bigramTransitionProbabilities = note_bigram_probability(midi_files)

    # Q4: Your code goes here
    # Can use regular numpy.log (i.e., natural logarithm)
    notes = note_extraction(midi_file)
    N = len(notes)

    log_probs = []
    log_probs.append(np.log(unigramProbabilities[notes[0]]))  # p(w_1|w_0) = p(w_1)

    for i in range(1, N):  # p(w_i|w_{i-1}) (i>1)
        prev_note = notes[i - 1]
        next_note = notes[i]
        index = bigramTransitions[prev_note].index(next_note)
        prob = bigramTransitionProbabilities[prev_note][index]
        log_probs.append(np.log(prob))

    perplexity = np.exp(-np.sum(log_probs) / N)
    return perplexity

5. Implement a second-order Markov chain, i.e., one which estimates p(next_note | next_previous_note, previous_note); write a function to compute the perplexity of this new model on a midi file.

    The perplexity of this model is defined as

    $\quad \text{exp}(-\frac{1}{N} \sum_{i=1}^N \text{log}(p(w_i|w_{i-2}, w_{i-1})))$

    where $p(w_1|w_{-1}, w_0) = p(w_1)$, $p(w_2|w_0, w_1) = p(w_2|w_1)$, $p(w_i|w_{i-2}, w_{i-1}) (i>2)$ refers to the probability p(next_note | next_previous_note, previous_note).


`note_trigram_probability()`
- **Input**: all midi files `midi_files`

- **Output**: two dictionaries:

  - `trigramTransitions`: key - (next_previous_note, previous_note), value - a list of next_note, e.g. {(60, 62):[64, 66, ..], (60, 64):[60, 64, ..], ...}

  - `trigramTransitionProbabilities`: key: (next_previous_note, previous_note), value: a list of probabilities for next_note in the same order of `trigramTransitions`, e.g. {(60, 62):[0.2, 0.2, ..], (60, 64):[0.4, 0.1, ..], ...}

`note_trigram_perplexity()`
- **Input**: a midi file

- **Output**: perplexity value

In [19]:
def note_trigram_probability(midi_files):
    trigramTransitions = defaultdict(list)
    trigramTransitionProbabilities = defaultdict(list)

    # Q5a: Your code goes here
    trigramTransitionsSets = defaultdict(set)
    trigram_counts = defaultdict(Counter)  # {(n_{i-2}, n_{i-1}): Counter({next_note: count})}
    for midi_file in midi_files:
        pitch_events = note_extraction(midi_file)
        for i in range(len(pitch_events) - 2):
            prev_notes = (pitch_events[i], pitch_events[i + 1])
            next_note = pitch_events[i + 2]
            trigramTransitionsSets[prev_notes].add(next_note)
            trigram_counts[prev_notes][next_note] += 1
    for (prev_notes, next_notes) in trigramTransitionsSets.items():
        trigramTransitions[prev_notes] = list(next_notes)

    for prev_notes, next_notes in trigramTransitions.items():
        total_notes = sum(trigram_counts[prev_notes].values())
        for next_note in next_notes:
            trigramTransitionProbabilities[prev_notes].append(trigram_counts[prev_notes][next_note] / total_notes)

    return trigramTransitions, trigramTransitionProbabilities

In [20]:
def note_trigram_perplexity(midi_file):
    unigramProbabilities = note_unigram_probability(midi_files)
    bigramTransitions, bigramTransitionProbabilities = note_bigram_probability(midi_files)
    trigramTransitions, trigramTransitionProbabilities = note_trigram_probability(midi_files)

    # Q5b: Your code goes here
    notes = note_extraction(midi_file)
    N = len(notes)

    log_probs = []
    # p(w_1|w_{-1}, w_0) = p(w_1)
    log_probs.append(np.log(unigramProbabilities[notes[0]]))
    # p(w_2|w_0, w_1) = p(w_2|w_1)
    prev_note = notes[0]
    next_note = notes[1]
    index = bigramTransitions[prev_note].index(next_note)
    log_probs.append(np.log(bigramTransitionProbabilities[prev_note][index]))

    for i in range(2, N):  # p(w_i|w_{i-2}, w_{i-1}) (i>2)
        prev_notes = (notes[i - 2], notes[i-1])
        next_note = notes[i]
        index = trigramTransitions[prev_notes].index(next_note)
        prob = trigramTransitionProbabilities[prev_notes][index]
        log_probs.append(np.log(prob))

    perplexity = np.exp(-np.sum(log_probs) / N)
    return perplexity

6. Our model currently doesn’t have any knowledge of beats. Write a function that extracts beat lengths and outputs a list of [(beat position; beat length)] values.

    Recall that each note will be encoded as `Position, Pitch, Velocity, Duration` using REMI. Please keep the `Position` value for beat position, and convert `Duration` to beat length using provided lookup table `duration2length` (see below).

    For example, for a note represented by four tokens `('Position_24', 'Pitch_72', 'Velocity_127', 'Duration_0.4.8')`, the extracted (beat position; beat length) value is `(24, 4)`.

    As a result, we will obtain a list like [(0,8),(8,16),(24,4),(28,4),(0,4)...], where the next beat position is the previous beat position + the beat length. As we divide each bar into 32 positions by default, when reaching the end of a bar (i.e. 28 + 4 = 32 in the case of (28, 4)), the beat position reset to 0.

In [21]:
duration2length = {
    '0.2.8': 2,  # sixteenth note, 0.25 beat in 4/4 time signature
    '0.4.8': 4,  # eighth note, 0.5 beat in 4/4 time signature
    '1.0.8': 8,  # quarter note, 1 beat in 4/4 time signature
    '2.0.8': 16, # half note, 2 beats in 4/4 time signature
    '4.0.4': 32, # whole note, 4 beats in 4/4 time signature
}

`beat_extraction()`
- **Input**: a midi file

- **Output**: a list of (beat position; beat length) values

In [22]:
def beat_extraction(midi_file):
    # Q6: Your code goes here
    midi = Score(midi_file)
    tokens = tokenizer(midi)[0].tokens

    beat_info = []
    current_position = 0

    i = 0
    while i < len(tokens):
        if tokens[i] == "Bar_None":
            current_position = 0
            i += 1
            continue

        position_idx = i
        # pitch_idx = i + 1
        # velocity_idx = i + 2
        duration_idx = i + 3

        beat_position = int(tokens[position_idx].split("_")[1])
        duration_str = tokens[duration_idx].split("_")[1]

        beat_length = duration2length[duration_str]
        beat_info.append((beat_position, beat_length))
        current_position = beat_position + beat_length

        if current_position >= 32:
            current_position = 0

        i += 4

    return beat_info

7. Implement a Markov chain that computes p(beat_length | previous_beat_length) based on the above function.

`beat_bigram_probability()`
- **Input**: all midi files `midi_files`

- **Output**: two dictionaries:

  - `bigramBeatTransitions`: key: previous_beat_length, value: a list of beat_length, e.g. {4:[8, 2, ..], 8:[8, 4, ..], ...}

  - `bigramBeatTransitionProbabilities`: key - previous_beat_length, value - a list of probabilities for beat_length in the same order of `bigramBeatTransitions`, e.g. {4:[0.3, 0.2, ..], 8:[0.4, 0.4, ..], ...}

In [23]:
def beat_pos_bigram_probability(midi_files):
    bigramBeatPosTransitions = defaultdict(list)
    bigramBeatPosTransitionProbabilities = defaultdict(list)

    # Q8a: Your code goes here
    bigramBeatPosTransitionsSets = defaultdict(set)
    beat_pos_counts = defaultdict(Counter)  # {position: Counter({length: count})}
    for midi_file in midi_files:
        beat_positions_lengths = beat_extraction(midi_file)
        for position, length in beat_positions_lengths:
            bigramBeatPosTransitionsSets[position].add(length)
            beat_pos_counts[position][length] += 1
    for (position, lengths) in bigramBeatPosTransitionsSets.items():
        bigramBeatPosTransitions[position] = list(lengths)

    for position, lengths in bigramBeatPosTransitions.items():
        total_lengths = sum(beat_pos_counts[position].values())
        for length in lengths:
            bigramBeatPosTransitionProbabilities[position].append(beat_pos_counts[position][length] / total_lengths)

    return bigramBeatPosTransitions, bigramBeatPosTransitionProbabilities

8. Implement a function to compute p(beat length | beat position), and compute the perplexity of your models from Q7 and Q8. For both models, we only consider the probabilities of predicting the sequence of **beat lengths**.

`beat_pos_bigram_probability()`
- **Input**: all midi files `midi_files`

- **Output**: two dictionaries:

  - `bigramBeatPosTransitions`: key - beat_position, value - a list of beat_length

  - `bigramBeatPosTransitionProbabilities`: key - beat_position, value - a list of probabilities for beat_length in the same order of `bigramBeatPosTransitions`

`beat_bigram_perplexity()`
- **Input**: a midi file

- **Output**: two perplexity values correspond to the models in Q7 and Q8, respectively

In [24]:
def beat_pos_bigram_probability(midi_files):
    bigramBeatPosTransitions = defaultdict(list)
    bigramBeatPosTransitionProbabilities = defaultdict(list)

    # Q8a: Your code goes here
    bigramBeatPosTransitionsSets = defaultdict(set)
    beat_pos_counts = defaultdict(Counter)  # {position: Counter({length: count})}
    for midi_file in midi_files:
        beat_positions_lengths = beat_extraction(midi_file)
        for position, length in beat_positions_lengths:
            bigramBeatPosTransitionsSets[position].add(length)
            beat_pos_counts[position][length] += 1
    for (position, lengths) in bigramBeatPosTransitionsSets.items():
        bigramBeatPosTransitions[position] = list(lengths)

    for position, lengths in bigramBeatPosTransitions.items():
        total_lengths = sum(beat_pos_counts[position].values())
        for length in lengths:
            bigramBeatPosTransitionProbabilities[position].append(beat_pos_counts[position][length] / total_lengths)

    return bigramBeatPosTransitions, bigramBeatPosTransitionProbabilities

In [25]:
def beat_bigram_perplexity(midi_file):
    bigramBeatTransitions, bigramBeatTransitionProbabilities = beat_bigram_probability(midi_files)
    bigramBeatPosTransitions, bigramBeatPosTransitionProbabilities = beat_pos_bigram_probability(midi_files)
    # Q8b: Your code goes here
    # Hint: one more probability function needs to be computed
    beat_positions_lengths = beat_extraction(midi_file)
    beat_positions = [position for position, _ in beat_positions_lengths]
    beat_lengths = [length for _, length in beat_positions_lengths]

    N = len(beat_positions_lengths)

    log_probs_Q7 = []
    log_probs_Q8 = []

    unigramLengthProbabilities = {}
    length_counts = Counter(beat_lengths)
    for length in length_counts.keys():
        unigramLengthProbabilities[length] = length_counts[length] / N
    prob = unigramLengthProbabilities[beat_lengths[0]]
    log_probs_Q7.append(np.log(prob))

    for i in range(N - 1):
        prev_length = beat_lengths[i]
        length = beat_lengths[i + 1]
        idx = bigramBeatTransitions[prev_length].index(length)
        prob = bigramBeatTransitionProbabilities[prev_length][idx]
        log_probs_Q7.append(np.log(prob))  # p(length | prev_length)

    for i in range(N):
        length = beat_lengths[i]
        position = beat_positions[i]
        idx = bigramBeatPosTransitions[position].index(length)
        prob = bigramBeatPosTransitionProbabilities[position][idx]
        log_probs_Q8.append(np.log(prob))  # p(length | position)

    # perplexity for Q7
    perplexity_Q7 = np.exp(-np.sum(log_probs_Q7) / len(log_probs_Q7))

    # perplexity for Q8
    perplexity_Q8 = np.exp(-np.sum(log_probs_Q8) / len(log_probs_Q8))

    return perplexity_Q7, perplexity_Q8

9. Implement a Markov chain that computes p(beat_length | previous_beat_length, beat_position), and report its perplexity.

`beat_trigram_probability()`
- **Input**: all midi files `midi_files`

- **Output**: two dictionaries:

  - `trigramBeatTransitions`: key: (previous_beat_length, beat_position), value: a list of beat_length

  - `trigramBeatTransitionProbabilities`: key: (previous_beat_length, beat_position), value: a list of probabilities for beat_length in the same order of `trigramBeatTransitions`

`beat_trigram_perplexity()`
- **Input**: a midi file

- **Output**: perplexity value

In [26]:
def beat_trigram_probability(midi_files):
    trigramBeatTransitions = defaultdict(list)
    trigramBeatTransitionProbabilities = defaultdict(list)

    # Q9a: Your code goes here
    for midi_file in midi_files:
        beat_positions_lengths = beat_extraction(midi_file)
        for i in range(len(beat_positions_lengths) - 1):
            prev_length = beat_positions_lengths[i][1]
            curr_position = beat_positions_lengths[i+1][0]
            curr_length = beat_positions_lengths[i+1][1]
            key = (prev_length, curr_position)
            trigramBeatTransitions[key].append(curr_length)

    for key, lengths in trigramBeatTransitions.items():
        length_counts = Counter(lengths)
        total_lengths = sum(length_counts.values())
        transitions = [length for length in length_counts.keys()]
        probs = [length_counts[length] / total_lengths for length in transitions]
        trigramBeatTransitions[key] = transitions
        trigramBeatTransitionProbabilities[key] = probs

    return trigramBeatTransitions, trigramBeatTransitionProbabilities

In [27]:
def beat_trigram_perplexity(midi_file):
    bigramBeatPosTransitions, bigramBeatPosTransitionProbabilities = beat_pos_bigram_probability(midi_files)
    trigramBeatTransitions, trigramBeatTransitionProbabilities = beat_trigram_probability(midi_files)
    # Q9b: Your code goes here
    beat_positions_lengths = beat_extraction(midi_file)

    N = len(beat_positions_lengths)
    log_probs = []

    position = beat_positions_lengths[0][0]
    length = beat_positions_lengths[0][1]
    index = bigramBeatPosTransitions[position].index(length)
    prob = bigramBeatPosTransitionProbabilities[position][index]
    log_probs.append(np.log(prob))

    for i in range(1, N):
        prev_length = beat_positions_lengths[i - 1][1]
        curr_position = beat_positions_lengths[i][0]
        curr_length = beat_positions_lengths[i][1]
        key = (prev_length, curr_position)
        index = trigramBeatTransitions[key].index(curr_length)
        prob = trigramBeatTransitionProbabilities[key][index]
        log_probs.append(np.log(prob))

    perplexity = np.exp(-np.sum(log_probs) / len(log_probs))
    return perplexity

10. Use the model from Q5 to generate N notes, and the model from Q8 to generate beat lengths for each note. Save the generated music as a midi file (see code from workbook1) as q10.mid. Remember to reset the beat position to 0 when reaching the end of a bar.

`music_generate`
- **Input**: target length, e.g. 500

- **Output**: a midi file q10.mid

Note: the duration of one beat in MIDIUtil is 1, while in MidiTok is 8. Divide beat length by 8 if you use methods in MIDIUtil to save midi files.

In [45]:
def music_generate(length):
    # sample notes
    unigramProbabilities = note_unigram_probability(midi_files)
    bigramTransitions, bigramTransitionProbabilities = note_bigram_probability(midi_files)
    trigramTransitions, trigramTransitionProbabilities = note_trigram_probability(midi_files)

    # Q10: Your code goes here ...
    sampled_notes = []
    notes = list(unigramProbabilities.keys())
    first_note = random.choices(notes, weights=unigramProbabilities.values())[0]
    sampled_notes.append(first_note)
    sampled_notes.append(sample_next_note(first_note))

    for _ in range(2, length):
        prev_notes = (sampled_notes[-2], sampled_notes[-1])
        if prev_notes in trigramTransitions:
            notes = trigramTransitions[prev_notes]
            probs = trigramTransitionProbabilities[prev_notes]
            sampled_notes.append(random.choices(notes, weights=probs)[0])
        else:
            print("dump 0")
            sampled_notes.append(random.choice(notes))

    # sample beats
    sampled_beats = []
    beat_pos_transitions, beat_pos_probabilities = beat_pos_bigram_probability(midi_files)
    # print(beat_pos_transitions.keys())
    current_position = 0
    for i in range(length):
        if current_position in beat_pos_transitions.keys():
            # print("not dump")
            beats = beat_pos_transitions[current_position]
            probs = beat_pos_probabilities[current_position]
            beat_length = random.choices(beats, weights=probs)[0]
        else:
            print("dump 1", current_position)
            beat_length = random.choice(list(duration2length.values()))
        sampled_beats.append((current_position, beat_length))
        current_position = (current_position + beat_length) % 32

    # save the generated music as a midi file
    midi = MIDIFile(1) # Create a MIDI file that consists of 1 track
    track = 0 # Set track number
    time = 0 # Where is the event placed (at the beginning)
    tempo = 120 # The tempo (beats per minute)
    midi.addTempo(track, time, tempo) # Add tempo information
    channel = 0
    volume = 100
    for i in range(length):
        pitch = sampled_notes[i]
        _, beat_length = sampled_beats[i]
        duration = beat_length / 8  # convert REMI to MIDI beat duration
        midi.addNote(track, channel, pitch, time, duration, volume)
        time += duration
    # add notes to MIDI file
    with open("q10.mid", "wb") as f:
        midi.writeFile(f)

In [46]:
# comment this out when submitting
music_generate(500)